# Importing Packages

In [1]:
from pyspark import SparkContext, SparkFiles
from pyspark.sql import SparkSession
import re
import os
from nltk.util import ngrams
import pandas as pd
import numpy as np
import pickle
import time
import operator
from fuzzywuzzy import fuzz

In [2]:
SparkContext.setSystemProperty('spark.executor.memory', '10g')
sc = SparkContext(appName='entity_resolution')

# Implementation

In [3]:
def pickle_out(name, obj, default_path='./sub_result/'):
    pickle_out = open(default_path + name + '.pickle','wb')
    pickle.dump(obj, pickle_out)
    pickle_out.close()
    return

def pickle_in(name, default_path='./sub_result/'):
    pickle_in = open(default_path + name + '.pickle','rb')
    obj = pickle.load(pickle_in)
    pickle_in.close()
    return obj

# Dictionaries for the asciifying task.
group_a = {'a':[u'\u00C0', u'\u00C1', u'\u00C2', u'\u00C3', u'\u00C4', u'\u00C5', u'\u00E0'
                 , u'\u00E1', u'\u00E2', u'\u00E3', u'\u00E4', u'\u00E5', u'\u0100', u'\u0101'
                 , u'\u0102', u'\u0103', u'\u0104', u'\u0105']}
group_c = {'c':[u'\u00C7', u'\u00E7', u'\u0106', u'\u0107', u'\u0108', u'\u0109', u'\u010A'
                 , u'\u010B', u'\u010C', u'\u010D']}
group_d = {'d':[u'\u00D0', u'\u00F0', u'\u010E', u'\u010F', u'\u0110', u'\u0111']}
group_e = {'e':[u'\u00C8', u'\u00C9', u'\u00CA', u'\u00CB', u'\u00E8', u'\u00E9', u'\u00EA'
                 , u'\u00EB', u'\u0112', u'\u0113', u'\u0114', u'\u0115', u'\u0116', u'\u0117'
                 , u'\u0118', u'\u0119', u'\u011A', u'\u011B']}
group_g = {'g':[u'\u011C', u'\u011D', u'\u011E', u'\u011F', u'\u0120', u'\u0121', u'\u0122'
                 , u'\u0123']}
group_h = {'h':[u'\u0124', u'\u0125', u'\u0126', u'\u0127']}
group_i = {'i':[u'\u00CC', u'\u00CD', u'\u00CE', u'\u00CF', u'\u00EC', u'\u00ED', u'\u00EE'
                 , u'\u00EF', u'\u0128', u'\u0129', u'\u012A', u'\u012B', u'\u012C', u'\u012D'
                 , u'\u012E', u'\u012F', u'\u0130', u'\u0131']}
group_j = {'j':[u'\u0134', u'\u0135']}
group_k = {'k':[u'\u0136', u'\u0137', u'\u0138']}
group_l = {'l':[u'\u0139', u'\u013A', u'\u013B', u'\u013C', u'\u013D', u'\u013E', u'\u013F'
                 , u'\u0140', u'\u0141', u'\u0142']}
group_n = {'n':[u'\u00D1', u'\u00F1', u'\u0143', u'\u0144', u'\u0145', u'\u0146', u'\u0147'
                 , '\u0148', '\u0149', '\u014A', '\u014B']}
group_o = {'o':[u'\u00D2', u'\u00D3', u'\u00D4', u'\u00D5', u'\u00D6', u'\u00D8', u'\u00F2'
                 , u'\u00F3', u'\u00F4', u'\u00F5', u'\u00F6', u'\u00F8', u'\u014C', u'\u014D'
                 , u'\u014E', u'\u014F', u'\u0150', u'\u0151']}
group_r = {'r':[u'\u0154', u'\u0155', u'\u0156', u'\u0157', u'\u0158', u'\u0159']}
group_s = {'s':[u'\u015A', u'\u015B', u'\u015C', u'\u015D', u'\u015E', u'\u015F', u'\u0160'
                 , u'\u0161', u'\u017F']}
group_t = {'t':[u'\u0162', u'\u0163', u'\u0164', u'\u0165', u'\u0166', u'\u0167']}
group_u = {'u':[u'\u00D9', u'\u00DA', u'\u00DB', u'\u00DC', u'\u00F9', u'\u00FA', u'\u00FB'
                 , u'\u00FC', u'\u0168', u'\u0169', u'\u016A', u'\u016B', u'\u016C', u'\u016D'
                 , u'\u016E', u'\u016F', u'\u0170', u'\u0171', u'\u0172', u'\u0173']}
group_w = {'w':[u'\u0174', u'\u0175']}
group_y = {'y':[u'\u00DD', u'\u00FD', u'\u00FF', u'\u0176', u'\u0177', u'\u0178']}
group_z = {'z':[u'\u0179', u'\u017A', u'\u017B', u'\u017C', u'\u017D', u'\u017E']}

group_list = [group_a, group_c, group_d, group_e, group_g, group_h, group_i, group_j,
             group_k, group_l, group_n, group_o, group_r, group_s, group_t, group_u,
             group_w, group_y, group_z]
translate_dict = {}
for group in group_list:
    return_value = group.keys()[0]
    for char in group[return_value]:
        translate_dict[char] = return_value
#sorted(translate_dict.items(), key=lambda kv: kv[1])
pickle_out('translate_dict', translate_dict)

def make_check_df_openrefine(series):
    result = []
    for row in series:
        temp_dict = {}
        choice_list = []
        for choice in row['choices']:
            choice_list.append(choice['v'])
        temp_dict['old_entity_open'] = set(choice_list)
        temp_dict['new_entity_open'] = row['value']
        result.append(temp_dict)
    result = pd.DataFrame(result)
    result.sort_values('new_entity_open', inplace=True)
    return result

In [24]:
class EntityResolution:
    """
    Authors: Myoungsu Choi (myoungsu@usc.edu), 
             Jingying Yin (yinjingy@usc.edu),
             Yining Zhang (yzhang27@usc.edu)
    """
    
    default_parameters = {
        'method': 'fingerprint',
        'partition_num': 8,
        'default_file_path': './sub_result/', 
        'signature_len': 15,
        'band_size': 5,
        'n': 2,
        'annotate': False
    }
    
    def __init__(self,
                 input_file_path=None,
                 partition_num=default_parameters['partition_num'],
                 method=default_parameters['method'],
                 signature_len=default_parameters['signature_len'],
                 band_size=default_parameters['band_size'],
                 default_file_path=default_parameters['default_file_path'],
                 annotate=default_parameters['annotate'],
                 n=default_parameters['n']):
        """
        Parameters
        ----------
        input_file_path: String, a path of the input file. 
        partition_num: Integer, a number of the partition for spark.
        method: String, a method for the entity resolution.
        signature_len: Integer, the row length of the signature matix.
        band_size: Integer, a band size of the LSH.
        default_file_path: String, a default file path for the pickling in and out.
        annotate: Boolean, whether to print the tree or not.
        """
        
        method_available = ['fingerprint', 'ngram_fingerprint',
                            'fingerprint_lhs', 'ngram_fingerprint_lhs']
        method_mapper = {
            'fingerprint':self.extract_fingerprint,
            'ngram_fingerprint':self.extract_ngram_fingerprint,
        }
        
        if not input_file_path:
            raise ValueError('Input_file_path should be given.')
        if method not in method_available:
            raise ValueError('Available methods are ' + str(method_available))
            
        self.method = method
        self.partition_num = partition_num
        self.input_file_path = input_file_path
        self.default_file_path = default_file_path
        self.signature_len = signature_len
        self.band_size = band_size
        self.banded_output = False
        translate_dict = self.pickle_in('translate_dict')
        
        self.func = method_mapper[self.method]
        self.translate_dict = translate_dict
        self.non_ascii_set = set(translate_dict.keys())
        self.n = n
        self.extra_char = "’"
        self.annotate = annotate
        
        if self.method == 'fingerprint':
            self.preprocess_func = self.make_tokens
        elif self.method == 'ngram_fingerprint':
            self.preprocess_func = self.make_ngrams
        
    def pickle_out(self, name, obj):
        """
        Parameters
        ----------
        name: String, a file name for the saving in pickle object.
        obj: Object, an object for the saving
        """
        if not os.path.exists(self.default_file_path):
            os.makedirs(self.default_file_path)
        pickle_out = open(self.default_file_path + name + '.pickle','wb')
        pickle.dump(obj, pickle_out)
        pickle_out.close()
        return

    def pickle_in(self, name):
        """
        Parameters
        ----------
        name: String, a file name for the saving in pickle object.
        
        Return
        ------
        A loaded object.
        """
        if not os.path.exists(self.default_file_path):
            raise ValueError(self.default_file_path+' is not existed.')
        pickle_in = open(self.default_file_path + name + '.pickle','rb')
        obj = pickle.load(pickle_in)
        pickle_in.close()
        return obj
    
    # Abstract funtion
    # Implemented in EntityResolutionLsh
    def make_tokens(self, string):
        return
    
    # Abstract funtion
    # Implemented in EntityResolutionLsh
    def make_ngrams(self, string):
        return
    
    def strip(self, string):
        """
        Parameters
        ----------
        string: String, a string for the strip task.
        
        Return
        ------
        A string after removing leading and trailing whitespace.
        """
        return string.strip()

    def lower(self, string):
        """
        Parameters
        ----------
        string: String, a string for the lower task.
        
        Return
        ------
        A string in a lowercase representation form
        """
        return string.lower()

    def remove_punctuation(self, string):
        """
        Parameters
        ----------
        string: String, a string for the removing punctuation task.
        
        Return
        ------
        A string after removing punctuation.
        """
        pattern = r'[^\w\s]'
        return re.sub(pattern, '', string)
    
    def remove_whitespace(self, string):
        """
        Parameters
        ----------
        string: String, a string for the removing whitespace task.
        
        Return
        ------
        A string after removing whitespace in any position of the string.
        """
        pattern = r'[\s]'
        return re.sub(pattern, '', string)
    
    def asciify(self, string):
        """
        Parameters
        ----------
        string: String, a string for the asciifying task.
        [Asciifying task: normalizing extended western characters to their ASCII representation]
        
        Return
        ------
        A string after asciifying task.
        """
        if not isinstance(string, unicode):
            string = unicode(string, 'utf-8')
        return_str = ''
        for char in string:
            if char in self.non_ascii_set:
                return_str += self.translate_dict[char]
            else:
                return_str += char
        return str(return_str)

    def tokenize(self, string):
        """
        Parameters
        ----------
        string: String, a string for the tokenizing task based on the whitespace.
        
        Return
        ------
        A list of the tokens after tokenizing.
        """
        return string.split()

    def remove_duplicates(self, token_list):
        """
        Parameters
        ----------
        token_list: List, a list for the removing the duplicates in the list.
        
        Return
        ------
        A list of the tokens after the removing dupliacates.
        """
        token_series = pd.Series(token_list)
        return list(token_series.unique())

    def sort_list(self, token_list):
        """
        Parameters
        ----------
        token_list: List, a list for the sorting task.
        
        Return
        ------
        A list of the tokens after the sorting task.
        """
        return sorted(token_list, reverse=False)

    def join_string(self, token_list, ngram=False):
        """
        Parameters
        ----------
        token_list: List, a list for the joining all the tokens based on the whitespace.
        ngram: Boolean, an indicator whether method is ngram based or not. 
        
        Return
        ------
        A string after the joining all the tokens based on whitespace.
        """
        if not ngram:
            return ' '.join(token_list)
        else:
            temp_list = []
            for tuple_item in token_list:
                for i in range(len(tuple_item)):
                    temp_list.append(tuple_item[i])
            return ''.join(temp_list)

    def get_ngram(self, token_list, n=2):
        """
        Parameters
        ----------
        token_list: List, a list for getting a ngram list.
        n: Integer, an number of near items gathered during the ngram.
        
        Return
        ------
        A list of the ngram items.
        """
        return list(ngrams(token_list, n))

    def extract_fingerprint(self, string):
        """
        Parameters
        ----------
        string: String, a string for the extracting a fingerprint.
        
        Return
        ------
        A fingerprint based on the fingerprinting method.
        """
        string = self.strip(string)
        string = self.lower(string)
        string = self.remove_punctuation(string)
        string = self.asciify(string)
        token_list = self.tokenize(string)
        token_list = self.sort_list(token_list)
        token_list = self.remove_duplicates(token_list)
        fingerprint = self.join_string(token_list)
        fingerprint = self.strip(fingerprint)
        return fingerprint

    def extract_ngram_fingerprint(self, string):
        """
        Parameters
        ----------
        string: String, a string for the extracting a fingerprint based on ngram.
        
        Return
        ------
        A fingerprint based on the ngram-fingerprinting method.
        """
        string = self.lower(string)
        string = self.remove_punctuation(string)
        string = self.remove_whitespace(string)
        ngram_list = self.get_ngram(list(string), n=self.n)
        ngram_list = self.sort_list(ngram_list)
        ngram_list = self.remove_duplicates(ngram_list)
        fingerprint = self.join_string(ngram_list, ngram=True)
        fingerprint = self.asciify(fingerprint)
        return fingerprint
    
    def get_normalized_entity(self, entity_list):
        """
        Parameters
        ----------
        entity_list: List, a list of the entities for the extracting fingerprint task.
        
        Return
        ------
        A list of the tuples of fingerprint and original entity string.
        """
        result = []
        for entity in entity_list:
            if not isinstance(entity, str):
                entity = str(entity)
            normalized_entity = self.func(entity)
            result.append((normalized_entity, entity))
        return result
    
    def count_by_values(self, group):
        temp_result = {}
        for item in group:
            if item in temp_result.keys():
                temp_result[item] += 1
            else:
                temp_result[item] = 1
        new_entity = max(temp_result.iteritems(), key=operator.itemgetter(1))[0]
        return (new_entity, temp_result)
    
    def clustering_entity(self, column=None, sc=None):
        """
        Parameters
        ----------
        column: String, a name of the column of the DataFrame, which is supposed to be worked on.
        sc: SparkContext instance.
        
        Return
        ------
        A DataFrame that includes 3 columns(fingerprint, original column, newly assigned column).
        """
        time1 = time.time()
        if not isinstance(column, str):
            raise ValueError('column should be string.')
        if not sc:
            raise ValueError('Spark Context should be given.')
        df_raw = pd.read_csv(self.input_file_path) 
        
        if column not in df_raw.columns:
            raise ValueError(column +' is not in the column list.')
        #df_raw[column] = df_raw[column].str.replace(self.extra_char, "'")
        #df_raw[column] = df_raw[column].str.replace(r'\s+', "\s")
        series_raw = df_raw[column]
        lines = sc.parallelize(series_raw, self.partition_num)
        
        entity_fingerprint_list = lines.mapPartitions(self.get_normalized_entity)
        result = entity_fingerprint_list.groupByKey()\
                 .mapValues(lambda x: self.count_by_values(x))\
                 .map(lambda x: (x[0], x[1][0], x[1][1])).collect()
        df_result = pd.DataFrame(result, columns=['fingerprint', 'new_entity', 'old_entities'])
        
        time2 = time.time()
        if self.annotate:
            print '• Function took %0.1f sec' % ((time2-time1))
            print '• '+ str(len(df_result[df_result['old_entities'].apply(lambda x: len(x)>1)]))\
                      + ' clusters founded.'
        return [(j, k.keys()) for i, j, k in result]
    
class EntityResolutionLsh(EntityResolution):
            
    def make_tokens(self, init_string):
        """
        Parameters
        ----------
        init_string: String, an original entity.
        
        Return
        ------
        A tuple of original string and a list of tokens.
        """
        string = self.strip(init_string)
        string = self.lower(string)
        string = self.remove_punctuation(string)
        string = self.asciify(string)
        token_list = self.tokenize(string)
        token_list = self.sort_list(token_list)
        token_list = self.remove_duplicates(token_list)
        return (init_string, token_list) 
    
    def make_ngrams(self, init_string):
        """
        Parameters
        ----------
        init_string: String, an original entity.
        
        Return
        ------
        A tuple of original string and a list of ngrams.
        """
        string = self.lower(init_string)
        string = self.remove_punctuation(string)
        string = self.remove_whitespace(string)
        ngram_list = self.get_ngram(list(string), n=self.n)
        ngram_list = self.sort_list(ngram_list)
        ngram_list = self.remove_duplicates(ngram_list)
        return (init_string, ngram_list)
    
    def get_preprocessed_entity(self, entity_list):
        """
        Parameters
        ----------
        entity_list: List, a list of the entities for the preprocessing task for the LSH task.
        
        Return
        ------
        A list of the tuples from the make_ngrams or the make_tokens function.
        """
        result = []
        for entity in entity_list:
            preprocessed_entity = self.preprocess_func(entity)
            result.append(preprocessed_entity)
        return result
    
    def hash_func(self, mid, i):
        """
        Parameters
        ----------
        mid: Integer, token or ngram id from the token2id.
        i: Integer, indices for the hashing [Scope: (1, signature_len)].
        
        Return
        ------
        An hashed integer. 
        """
        return (3*mid + 11*i) % 100 + 1

    def calc_jaccard(self, list1, list2):
        """
        Parameters
        ----------
        list1, list2: List, lists for the calculating jaccard score.
        
        Return
        ------
        A jaccard score.
        """
        set1 = set(list1)
        set2 = set(list2)
        union = set1 | set2
        intersect = set1 & set2
        return len(intersect)/len(union)
    
    def band2idx(self, band_num, band_size, signature_len):
        """
        Parameters
        ----------
        band_num: Integer, an index of the band.
        band_size: Integer, the size of the band for the LSH task.
        signature_len: Integer, the row length of the signature matix.
        
        Return
        ------
        A tuple of start index and end index for the making banded minhash matrix.
        """
        if band_num > band_size:
            raise ValueError('band_num cannot be larger than band_size')
        unit = signature_len/band_size
        start_idx = band_num*unit
        end_idx = start_idx + unit
        return (start_idx, end_idx)
    
    def minhash(self, iterator):
        """
        Parameters
        ----------
        iterator: List, 
                  a list of (entity, [token1, token2, ,,,]) or (entity, [ngram1, ngram2, ,,,])
                  characteristic matrix
        Return
        ======
        A list of the tuples. (original entity, a signature column[list type])
        • banded_output: (band_index, (original entity, a signature column[list type]))
        """
        result = []
        for entity in iterator:
            signature_col = np.full(self.signature_len, np.inf)
            for token in entity[1]:
                for i in range(self.signature_len):
                    hashed_idx = self.hash_func(self.token2id[token], i+1)
                    if hashed_idx < signature_col[i]:
                        signature_col[i] = hashed_idx
            if self.banded_output:
                for band in range(self.band_size):
                    start_idx, end_idx = self.band2idx(band, 
                                                       self.band_size, 
                                                       self.signature_len)
                    result.append((band, (entity[0], list(signature_col)[start_idx:end_idx])))
            else:
                result.append((entity[0], list(signature_col)))
        return result

    def make_bucket(self, iterator):
        """
        Parameters
        ----------
        iterator: List, a banded minhash matrix.
        
        Return
        ======
        A list of the tuples of band_num and 
        a dictionary of (key:hashed_value, value: a list of entities that shares the same hash.)
        """
        result = []
        for band in iterator:
            itermediate_dict = {}
            for item in band[1]:
                hashed_value = hash(str(item[1]))
                if not hashed_value in itermediate_dict.keys():
                    itermediate_dict[hashed_value] = set([item[0]])
                else:
                    itermediate_dict[hashed_value] = itermediate_dict[hashed_value] | set([item[0]])
            result.append((band[0], itermediate_dict))
        return result

    def find_candidates(self, band, entity):
        """
        Parameters
        ----------
        band: Integer, an index of the band
        entity: String, the original entity.
        
        Return
        ======
        A list of candidates for the similar entities. 
        """
        candidates_list = set()
        for similar_entity_set in band[1].values():
            if entity in similar_entity_set:
                similar_entity_set = similar_entity_set - set([entity])
                candidates_list = candidates_list | similar_entity_set
        return list(candidates_list)

    def find_similar_entity(self, 
                            entity,
                            candidates_list, 
                            minhash_matrix, 
                            top=5):
        """
        Parameters
        ----------
        entity: String, the original entity.
        candidates_list: List, a list of candidates for the similar entities. 
        minhash_matrix: List, a list of the tuples. 
                        (original entity, a signature column[list type])
        top: Integer, a number of how many similar entities will be returned.
        
        Return
        ======
        A list of similar entities.
        """
        similar_entity_list = []
        try:
            if candidates_list <= top:
                return candidates_list
            else:
                entity_col = minhash_matrix[entity]
                
                for cand in candidates_list:
                    if cand:
                        candidate_col = minhash_matrix[cand]
                        jaccard_score = self.calc_jaccard(entity_col, candidate_col)
                        similar_entity_list.append((jaccard_score, cand))
                similar_entity_list = sorted(similar_entity_list, reverse=True)
                similar_entity_list = [entity for score, entity in similar_entity_list]
        except KeyError as error:
            return similar_entity_list[:top]
        return similar_entity_list[:top]
    
    def make_token2id_id2token(self, preprocessed_lines):
        """
        Parameters
        ----------
        preprocessed_lines: List, a list of the tuples (init_string, token_list or ngram_list)
                            This is from self.preprocess_func 
                            [self.make_tokens or self.make_ngrams]
        
        """
        all_tokens = []
        for item in preprocessed_lines:
            all_tokens.extend(item[1])
        all_tokens = pd.Series(all_tokens).unique()
        self.token2id = {item:i for i, item in enumerate(all_tokens)}
        self.id2token = {i:item for i, item in enumerate(all_tokens)}
        return 
    
    def predict(self, entity_list):
        """
        Parameters
        ----------
        entity_list: List, a list of the original entities that are supposed to be worked on.
        
        Return
        ======
        A list of the tuples of original entity and newly assigned entity.
        """
        result = []
        for entity in entity_list:
            candidates_list = []
            for band in self.buckets:
                candidates_list.extend(self.find_candidates(band, entity))
            candidates_list = list(set(candidates_list))
            similar_entity = self.find_similar_entity(entity, 
                                                      candidates_list, 
                                                      self.minhash_matrix)
            result.append((entity, similar_entity))
        return result
    
    def fuzzy_dist(self, interator):
        result = []
        for entity, lt in interator:
            max_val = 0
            max_ent = None
            for ent in lt:
                ratio = fuzz.ratio(entity, ent)
                if ratio > max_val:
                    max_val = ratio
                    max_ent = ent
            result.append((entity, max_ent, max_val))
        return result
    
    def clustering_entity(self, column=None, sc=None):
        """
        Parameters
        ----------
        column: String, a name of the column of the DataFrame, which is supposed to be worked on.
        sc: SparkContext.
        
        Return
        ======
        A list of the tuples of original entity and newly assigned entity.
        """
        time1 = time.time()
        if not isinstance(column, str):
            raise ValueError('column should be string.')
        if not sc:
            raise ValueError('Spark Context should be given.')
            
        df_raw = pd.read_csv(self.input_file_path) 
        if column not in df_raw.columns:
            raise ValueError(column +' is not in the column list.')
        #df_raw[column] = df_raw[column].str.replace(self.extra_char, "'")
        series_raw = df_raw[column]
        lines = sc.parallelize(series_raw, self.partition_num)
        
        entity_fingerprint_list = lines.mapPartitions(self.get_normalized_entity)
        preprocessed_lines = entity_fingerprint_list.map(lambda x: (x[1], x[0].split())).collect()
        #preprocessed_lines = preprocessed_lines.mapPartitions(self.get_preprocessed_entity).collect()
        self.make_token2id_id2token(preprocessed_lines)
        
        prep_lines_serial = sc.parallelize(preprocessed_lines, self.partition_num)
        
        self.banded_output = True
        minhash_banded_matrix = prep_lines_serial\
                                .mapPartitions(self.minhash)\
                                .collect()
        self.banded_output = False
        minhash_matrix = prep_lines_serial\
                        .mapPartitions(self.minhash).collect()
        minhash_matrix = {entity:column for entity, column in minhash_matrix}
        self.minhash_matrix = minhash_matrix
        
        buckets = sc.parallelize(minhash_banded_matrix, self.partition_num)
        buckets = buckets.groupByKey().map(lambda x: (x[0], list(x[1])))
        buckets = buckets.mapPartitions(self.make_bucket).collect()
        self.buckets = buckets
        
        pred_result = sc.parallelize(series_raw, self.partition_num)
        pred_result = pred_result.mapPartitions(self.predict)\
                      .collect()
        result = sc.parallelize(pred_result, self.partition_num).collect()
        
        result = self.fuzzy_dist(result)
        result = sorted(result, key=lambda kv: kv[2], reverse=True)
        time2 = time.time()
        if self.annotate:
            print '• Function took %0.1f sec' % ((time2-time1))
        return [(entity, sim) for entity, sim, score in result]
               

input_file_path_la = './data/restaurant-and-market-health-inspections.csv'
input_file_path_ny = './data/newyork-restaurant.csv'
#input_file_path_ny = './data/newyork.csv'
#sample_file_path = './data/sample.csv'

# df_ratings = pd.read_csv(input_file_path)
# df_ratings_sample = df_ratings.sample(10000)
# df_ratings_sample.to_csv(sample_file_path, index=False)
# df_sample = pd.read_csv(sample_file_path)

In [21]:
df_ny = pd.read_csv(input_file_path_la)

In [23]:
len(df_ny)

67573

In [13]:
df_ny = pd.read_csv('./data/newyork.csv')
df_ny = df_ny.sample(frac=0.3)
df_ny.to_csv('newyork-restaurant.csv', index=False)

In [29]:
er_lsh = EntityResolutionLsh(input_file_path=input_file_path_la, 
                             method='fingerprint',
                             partition_num=16,
                             signature_len=30,
                             band_size=10,
                             annotate=True,
                             n=2)
result_la_lsh_f = er_lsh.clustering_entity(column='facility_name', sc=sc)

• Function took 153.6 sec


In [37]:
result_la_lsh_f

[('JACK IN THE BOX # 3215', 'JACK IN THE BOX #3215'),
 ('CHIPOTLE MEXICAN GRILL', 'CHIPOTLE  MEXICAN GRILL'),
 ('LEONARDOS RESTAURANT', "LEONARDO'S RESTAURANT"),
 ('CHIPOTLE MEXICAN GRILL', 'CHIPOTLE  MEXICAN GRILL'),
 ('CHIPOTLE MEXICAN GRILL', 'CHIPOTLE  MEXICAN GRILL'),
 ('JACK IN THE BOX # 3343', 'JACK IN THE BOX #3343'),
 ("LEONARDO'S RESTAURANT", 'LEONARDOS RESTAURANT'),
 ('SIZZLER USA RESTAURANTS, INC', 'SIZZLER USA RESTAURANTS INC'),
 ('CHIPOTLE MEXICAN GRILL', 'CHIPOTLE  MEXICAN GRILL'),
 ('CHIPOTLE MEXICAN GRILL', 'CHIPOTLE  MEXICAN GRILL'),
 ('CHIPOTLE MEXICAN GRILL', 'CHIPOTLE  MEXICAN GRILL'),
 ('CHIPOTLE  MEXICAN GRILL', 'CHIPOTLE MEXICAN GRILL'),
 ('MOTHERS NUTRITIONAL CENTER', "MOTHER'S NUTRITIONAL CENTER"),
 ('SIZZLER USA RESTAURANTS INC', 'SIZZLER USA RESTAURANTS, INC'),
 ('YANG BAN SUL LUNG TANG INC', 'YANG BA SUL LUNG TANG INC'),
 ("FORD'S FILLING STATION", 'FORDS FILLING STATION'),
 ('CHIPOTLE MEXICAN GRILL', 'CHIPOTLE  MEXICAN GRILL'),
 ("FORD'S FILLING STATION", 

In [30]:
er_lsh = EntityResolutionLsh(input_file_path=input_file_path_la, 
                             method='ngram_fingerprint',
                             partition_num=16,
                             signature_len=30,
                             band_size=10,
                             annotate=True,
                             n=2)
result_la_lsh_n = er_lsh.clustering_entity(column='facility_name', sc=sc)

• Function took 79.5 sec


In [36]:
result_la_lsh_n

[("WETZELS PRETZEL'S", 'WETZELS PRETZELS'),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ('WETZELS PRETZELS', "WETZELS PRETZEL'S"),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ('WETZELS PRETZELS', "WETZELS PRETZEL'S"),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ('WINGSTOP REST #3', 'WING STOP REST #3'),
 ('WETZELS PRETZELS', "WETZELS PRETZEL'S"),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ('WETZELS PRETZELS', "WETZELS PRETZEL'S"),
 ('WING STOP REST #3', 'WINGSTOP REST #3'),
 ("WETZEL'S PRETZELS", 'WETZELS PRETZELS'),
 ('WETZELS PRETZELS', "WETZELS P

In [31]:
er = EntityResolution(input_file_path=input_file_path_la, 
                      method='fingerprint',
                      partition_num=16,
                      annotate=True,
                      n=3)
result_la_f = er.clustering_entity(column='facility_name', sc=sc)

• Function took 7.5 sec
• 107 clusters founded.


In [32]:
er = EntityResolution(input_file_path=input_file_path_la, 
                      method='ngram_fingerprint',
                      partition_num=16,
                      annotate=True,
                      n=2)
result_la_n = er.clustering_entity(column='facility_name', sc=sc)

• Function took 8.2 sec
• 157 clusters founded.


In [33]:
pickle_out('result_la_lsh_f', result_la_lsh_f)
pickle_out('result_la_lsh_n', result_la_lsh_n)
pickle_out('result_la_f', result_la_f)
pickle_out('result_la_n', result_la_n)

In [13]:
er_lsh = EntityResolutionLsh(input_file_path=input_file_path_ny, 
                             method='fingerprint',
                             partition_num=16,
                             signature_len=30,
                             band_size=10,
                             annotate=True,
                             n=2)
result_ny_lsh_f = er_lsh.clustering_entity(column='DBA', sc=sc)

• Function took 504.2 sec


In [38]:
result_ny_lsh_f

[('BIRDBATH NEIGHBORHOOD GREEN BAKERY', 'BIRDBATH  NEIGHBORHOOD GREEN BAKERY'),
 ('BIRDBATH NEIGHBORHOOD GREEN BAKERY', 'BIRDBATH  NEIGHBORHOOD GREEN BAKERY'),
 ('BIRDBATH NEIGHBORHOOD GREEN BAKERY', 'BIRDBATH  NEIGHBORHOOD GREEN BAKERY'),
 ('BIRDBATH  NEIGHBORHOOD GREEN BAKERY', 'BIRDBATH NEIGHBORHOOD GREEN BAKERY'),
 ('BIRDBATH NEIGHBORHOOD GREEN BAKERY', 'BIRDBATH  NEIGHBORHOOD GREEN BAKERY'),
 ('BIRDBATH  NEIGHBORHOOD GREEN BAKERY', 'BIRDBATH NEIGHBORHOOD GREEN BAKERY'),
 ('BIRDBATH NEIGHBORHOOD GREEN BAKERY', 'BIRDBATH  NEIGHBORHOOD GREEN BAKERY'),
 ('BIRDBATH NEIGHBORHOOD GREEN BAKERY', 'BIRDBATH  NEIGHBORHOOD GREEN BAKERY'),
 ('BIRDBATH NEIGHBORHOOD GREEN BAKERY', 'BIRDBATH  NEIGHBORHOOD GREEN BAKERY'),
 ('BIRDBATH NEIGHBORHOOD GREEN BAKERY', 'BIRDBATH  NEIGHBORHOOD GREEN BAKERY'),
 ('BIRDBATH  NEIGHBORHOOD GREEN BAKERY', 'BIRDBATH NEIGHBORHOOD GREEN BAKERY'),
 ("DUNKIN' DONUTS, BASKIN ROBBINS", "DUNKIN' DONUTS BASKIN ROBBINS"),
 ("DUNKIN' DONUTS, BASKIN ROBBINS", "DUNKIN' DONUT

In [14]:
er_lsh = EntityResolutionLsh(input_file_path=input_file_path_ny, 
                             method='ngram_fingerprint',
                             partition_num=16,
                             signature_len=30,
                             band_size=10,
                             annotate=True,
                             n=2)
result_ny_lsh_n = er_lsh.clustering_entity(column='DBA', sc=sc)

• Function took 243.3 sec


In [39]:
result_ny_lsh_n

[("WOLFGANG'S STEAKHOUSE", "WOLFGANG'S STEAK HOUSE"),
 ("WOLFGANG'S STEAKHOUSE", "WOLFGANG'S STEAK HOUSE"),
 ("WOLFGANG'S STEAKHOUSE", "WOLFGANG'S STEAK HOUSE"),
 ("WOLFGANG'S STEAKHOUSE", "WOLFGANG'S STEAK HOUSE"),
 ("WOLFGANG'S STEAKHOUSE", "WOLFGANG'S STEAK HOUSE"),
 ("WOLFGANG'S STEAKHOUSE", "WOLFGANG'S STEAK HOUSE"),
 ('YIN YANG COFFEE & TEA', 'YINYANG COFFEE & TEA'),
 ("WOLFGANG'S STEAKHOUSE", "WOLFGANG'S STEAK HOUSE"),
 ("WOLFGANG'S STEAKHOUSE", "WOLFGANG'S STEAK HOUSE"),
 ("WOLFGANG'S STEAK HOUSE", "WOLFGANG'S STEAKHOUSE"),
 ("WOLFGANG'S STEAKHOUSE", "WOLFGANG'S STEAK HOUSE"),
 ('YIN YANG COFFEE & TEA', 'YINYANG COFFEE & TEA'),
 ("WOLFGANG'S STEAKHOUSE", "WOLFGANG'S STEAK HOUSE"),
 ('YIN YANG COFFEE & TEA', 'YINYANG COFFEE & TEA'),
 ("WOLFGANG'S STEAKHOUSE", "WOLFGANG'S STEAK HOUSE"),
 ("WOLFGANG'S STEAKHOUSE", "WOLFGANG'S STEAK HOUSE"),
 ("WOLFGANG'S STEAKHOUSE", "WOLFGANG'S STEAK HOUSE"),
 ('YIN YANG COFFEE & TEA', 'YINYANG COFFEE & TEA'),
 ("WOLFGANG'S STEAKHOUSE", "WOLFGANG

In [25]:
er = EntityResolution(input_file_path=input_file_path_ny, 
                      method='fingerprint',
                      partition_num=16,
                      annotate=True,
                      n=2)
result_ny_f = er.clustering_entity(column='DBA', sc=sc)

• Function took 42.8 sec
• 232 clusters founded.


In [26]:
er = EntityResolution(input_file_path=input_file_path_ny, 
                      method='ngram_fingerprint',
                      partition_num=16,
                      annotate=True,
                      n=2)
result_ny_n = er.clustering_entity(column='DBA', sc=sc)

• Function took 45.2 sec
• 270 clusters founded.


In [28]:
pickle_out('result_ny_lsh_f', result_ny_lsh_f)
pickle_out('result_ny_lsh_n', result_ny_lsh_n)
pickle_out('result_ny_f', result_ny_f)
pickle_out('result_ny_n', result_ny_n)

In [373]:
def make_check_df_team(result):
    # new and old
    result_dict = {}
    for new, old in result:
        if new in result_dict.keys():
            result_dict[new] = result_dict[new] | set(old)
        else:
            result_dict[new] = set(old)
    keys = result_dict.keys()
    vals = result_dict.values()
    output = pd.DataFrame({
        'new_entity_team': keys,
        'old_entity_team': vals
    })
    result_team = output[output['old_entity_team'].apply(lambda x: len(x)>1)]
    return result_team

In [335]:
df_new = pd.read_csv('./data/newyork.csv')

In [337]:
df_new.columns

Index([u'CAMIS', u'DBA', u'BORO', u'BUILDING', u'STREET', u'ZIPCODE', u'PHONE',
       u'CUISINE DESCRIPTION', u'INSPECTION DATE', u'ACTION',
       u'VIOLATION CODE', u'VIOLATION DESCRIPTION', u'CRITICAL FLAG', u'SCORE',
       u'GRADE', u'GRADE DATE', u'RECORD DATE', u'INSPECTION TYPE'],
      dtype='object')

In [338]:
df_new.head(3)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,06/15/2015,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140Âº F.,Critical,30.0,NaN,NaN,08/28/2017,Cycle Inspection / Initial Inspection
1,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,11/25/2014,Violations were cited in the following area(s).,20F,Current letter grade card not posted.,Not Critical,NaN,NaN,NaN,08/28/2017,Administrative Miscellaneous / Initial Inspection
2,50046354,VITE BAR,QUEENS,2507,BROADWAY,11106.0,3478134702,Italian,10/03/2016,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,2.0,NaN,NaN,08/28/2017,Pre-permit (Operational) / Initial Inspection


In [330]:
result_team = make_check_df_team(result)

In [331]:
df_open_refine_raw = pd.read_json('./data/open_refine.json')
df_open_refine_raw = df_open_refine_raw[['clusters']]
result_open = make_check_df_openrefine(df_open_refine_raw['clusters'])

In [332]:
def measure_difference(result_team, result_open):
    result_team.sort_values('new_entity_team', inplace=True)
    result_team.set_index(pd.Index(range(len(result_team))), inplace=True)
    result_team['new_entity_team'] = \
        result_team['new_entity_team'].apply(lambda x: x.encode('utf-8'))
    result_open.sort_values('new_entity_open', inplace=True)
    result_open.set_index(pd.Index(range(len(result_open))), inplace=True)
    result_open['new_entity_open'] = \
        result_open['new_entity_open'].apply(lambda x: x.encode('utf-8'))
    joined_result = pd.merge(result_team, result_open,
                            how='left', left_index=True, right_index=True)
    joined_result['distance'] = \
    joined_result.apply(lambda x: fuzz.ratio(x.new_entity_team, x.new_entity_open), axis=1)
    return joined_result

In [334]:
result_team.head(4)

,new_entity_team,old_entity_team
0,23RD ST CAFE,"{23RD ST. CAFE, 23RD ST CAFE}"
1,5 STAR MARKET,"{5 STAR MARKET, 5- STAR- MARKET}"
2,7 - ELEVEN,"{7- ELEVEN, 7 ELEVEN, 7 - ELEVEN}"
3,AL'S LIQUOR,"{AL'S LIQUOR, ALS LIQUOR}"


In [317]:
df_open_refine_raw = pd.read_json('./data/open_refine.json')
df_open_refine_raw = df_open_refine_raw[['clusters']]
result_open = make_check_df_openrefine(df_open_refine_raw['clusters'])

In [287]:
result_open

,new_entity_open,old_entity_open
96,23RD ST CAFE,"{23RD ST. CAFE, 23RD ST CAFE}"
15,5 STAR MARKET,"{5 STAR MARKET, 5- STAR- MARKET}"
2,7 - ELEVEN,"{7- ELEVEN, 7 ELEVEN, 7 - ELEVEN}"
20,AL'S LIQUOR,"{ALS LIQUOR, AL'S LIQUOR}"
77,AMIGOS MARKET,"{AMIGOS MARKET, AMIGO'S MARKET}"
95,AROMA CAFE,"{CAFE AROMA, AROMA CAFE}"
22,BBQ CHICKEN LA,"{BBQ CHICKEN L.A., BBQ CHICKEN LA}"
53,BIG MAMA'S & PAPA'S PIZZERIA,"{BIG MAMAS & PAPAS PIZZERIA, BIG MAMA'S & PAPA..."
27,BIONICOS DAISYS,"{BIONICOS DAISYS, BIONICOS DAISY'S}"
82,CAFE ANZIO,"{ANZIO CAFE, CAFE ANZIO}"


In [198]:
result

[('PIZZA ITALIA', 'PIZZA ITALIA'),
 ('SMITH PHARMACY', 'SMITH PHARMACY'),
 ('BIG FATPITA CENTURY', 'BIG FATPITA CENTURY'),
 ('STARBUCKS COFFEE', 'STARBUCKS COFFEE'),
 ('GUATEMALA RESTAURANT', 'GUATEMALA RESTAURANT'),
 ('SPROUTS FARMERS MARKET', 'SPROUTS FARMERS MARKET'),
 ('RITE AID #5439', 'RITE AID #5439'),
 ('TOMAS MARKET', 'TOMAS MARKET'),
 ("LITO'S CORK ROOM", "LITO'S CORK ROOM"),
 ('OYSTARS', 'OYSTARS'),
 ('AMISTAD CAFE', 'AMISTAD CAFE'),
 ('DRIPS & SWIRLS', 'DRIPS & SWIRLS'),
 ('LAS MOLENDERAS', 'LAS MOLENDERAS'),
 ('SOREGASHI', 'SOREGASHI'),
 ('KING FOOD', 'KING FOOD'),
 ('TANDOORI EATS', 'TANDOORI EATS'),
 ('SERVED 2 ENJOY', 'SERVED 2 ENJOY'),
 ('MARVISTA RANCH MARKET', 'MARVISTA RANCH MARKET'),
 ("BENNY'S MARKET", "BENNY'S MARKET"),
 ('AGUA ARCO IRIS', 'AGUA ARCO IRIS'),
 ('SUBWAY  60937', 'SUBWAY  60937'),
 ('UNITED FAMILY ARCO #9637', 'UNITED FAMILY ARCO #9637'),
 ('MEAN GREENS', 'MEAN GREENS'),
 ('LA POUBELLE', 'LA POUBELLE'),
 ('ACOSTA MARKET', 'ACOSTA MARKET'),
 ('ADAMS 

In [115]:
df_team = make_check_df_team(result)

In [207]:
check_list = df_open_refine['choices'].values
for item in df_result['choices'].values:
    if not item in check_list:
        print(item)

set(['MCDONALD\xe2\x80\x99S #1126', "MCDONALD'S  #1126"])


In [208]:
check_list = df_result['choices'].values
for item in df_open_refine['choices'].values:
    if not item in check_list:
        print(item)

#### Difference
    - [1 item]: [MCDONALD'S  #1126]<->[MCDONALD’S #1126] 

In [209]:
a = [('a', 'c'), ('a', 'd'), ('a', 'e'), ('a', 'c')]

In [210]:
lines = sc.parallelize(a)

In [228]:
lines.groupByKey().mapValues(lambda x: count_by_values(x)).collect()

[('a', ('c', {'c': 2, 'd': 1, 'e': 1}))]

In [230]:
lines.collect()

[('a', 'c'), ('a', 'd'), ('a', 'e'), ('a', 'c')]

In [6]:
df_test = pd.read_csv('./data/restaurant-and-market-health-inspections.csv')

In [9]:
result = df_test.groupby('facility_name')['facility_address'].nunique()

In [11]:
m_f_names = result[result>1].index.to_list()

In [14]:
one_names = result[~result.index.isin(m_f_names)].index

In [17]:
df_test[df_test['facility_name']=='DODGER STADIUM']

,serial_number,activity_date,facility_name,score,grade,service_code,service_description,employee_id,facility_address,facility_city,facility_id,facility_state,facility_zip,owner_id,owner_name,pe_description,program_element_pe,program_name,program_status,record_id
3482,DAMMZ4VQK,2017-09-25T00:00:00.000,DODGER STADIUM,93,A,1,ROUTINE INSPECTION,EE0000122,1000 VIN SCULLY AVE,LOS ANGELES,FA0019271,CA,90012,OW0021606,LEVY PREMIUM FOOD SERVICE LP,RESTAURANT (151 + ) SEATS LOW RISK,1639,STADIUM CLUB BAR,ACTIVE,PR0199916
3493,DA3V15MEO,2017-09-25T00:00:00.000,DODGER STADIUM,94,A,401,OWNER INITIATED ROUTINE INSPECT.,EE0000122,1000 VIN SCULLY AVE,LOS ANGELES,FA0019271,CA,90012,OW0021606,LEVY PREMIUM FOOD SERVICE LP,RESTAURANT (151 + ) SEATS HIGH RISK,1641,STADIUM CLUB,ACTIVE,PR0037545
4102,DABRCQSZY,2017-09-08T00:00:00.000,DODGER STADIUM,85,B,1,ROUTINE INSPECTION,EE0000122,1000 VIN SCULLY AVE,LOS ANGELES,FA0019271,CA,90012,OW0021606,LEVY PREMIUM FOOD SERVICE LP,RESTAURANT (151 + ) SEATS HIGH RISK,1641,STADIUM CLUB,ACTIVE,PR0037545
4108,DAM0WRGYL,2017-09-08T00:00:00.000,DODGER STADIUM,96,A,1,ROUTINE INSPECTION,EE0000122,1000 VIN SCULLY AVE,LOS ANGELES,FA0019271,CA,90012,OW0021606,LEVY PREMIUM FOOD SERVICE LP,RESTAURANT (151 + ) SEATS HIGH RISK,1641,#952 TOP DECK DOGS TOO,ACTIVE,PR0012340
4123,DAFYEBDT0,2017-09-07T00:00:00.000,DODGER STADIUM,94,A,1,ROUTINE INSPECTION,EE0000122,1000 VIN SCULLY AVE,LOS ANGELES,FA0019271,CA,90012,OW0021606,LEVY PREMIUM FOOD SERVICE LP,RESTAURANT (0-30) SEATS HIGH RISK,1632,#210 LA TAQUERIA,ACTIVE,PR0006573
4172,DAYZCQZ4M,2017-09-07T00:00:00.000,DODGER STADIUM,95,A,1,ROUTINE INSPECTION,EE0000122,1000 VIN SCULLY AVE,LOS ANGELES,FA0019271,CA,90012,OW0021606,LEVY PREMIUM FOOD SERVICE LP,RESTAURANT (0-30) SEATS HIGH RISK,1632,#208 CHICK N TOTS,ACTIVE,PR0042380
4179,DANDCRBB3,2017-09-07T00:00:00.000,DODGER STADIUM,90,A,1,ROUTINE INSPECTION,EE0000122,1000 VIN SCULLY AVE,LOS ANGELES,FA0019271,CA,90012,OW0021606,LEVY PREMIUM FOOD SERVICE LP,RESTAURANT (0-30) SEATS HIGH RISK,1632,PRESS AREA DINING,ACTIVE,PR0008244
4191,DAUL5VU89,2017-09-06T00:00:00.000,DODGER STADIUM,98,A,1,ROUTINE INSPECTION,EE0000122,1000 VIN SCULLY AVE,LOS ANGELES,FA0019271,CA,90012,OW0021606,LEVY PREMIUM FOOD SERVICE LP,RESTAURANT (0-30) SEATS HIGH RISK,1632,#739 DODGER DOGS EXPRESS,ACTIVE,PR0044474
4193,DAWHIQ9YO,2017-09-06T00:00:00.000,DODGER STADIUM,90,A,1,ROUTINE INSPECTION,EE0000122,1000 VIN SCULLY AVE,LOS ANGELES,FA0019271,CA,90012,OW0021606,LEVY PREMIUM FOOD SERVICE LP,RESTAURANT (151 + ) SEATS HIGH RISK,1641,DUGOUT CLUB - IN SEAT KITCHEN,ACTIVE,PR0194097
4206,DA2FEPL9I,2017-09-06T00:00:00.000,DODGER STADIUM,91,A,1,ROUTINE INSPECTION,EE0000122,1000 VIN SCULLY AVE,LOS ANGELES,FA0019271,CA,90012,OW0021606,LEVY PREMIUM FOOD SERVICE LP,RESTAURANT (151 + ) SEATS HIGH RISK,1641,#741 LA TAQUERIA & LA TAQUERIA EXPRESS,ACTIVE,PR0009522


In [25]:
a = df_test['record_id'].value_counts()

In [27]:
a[a>1][:5]

PR0037545    12
PR0164507    12
PR0122106    12
PR0122758    11
PR0009367    11
Name: record_id, dtype: int64

In [30]:
len(df_test['facility_name'].unique())

12071

In [40]:
df_test.groupby('record_id')['facility_address'].nunique()[df_test.groupby('record_id')['facility_address'].nunique()>1]

record_id
PR0030008    2
PR0038384    2
PR0040941    2
PR0045771    2
PR0129590    2
PR0151401    2
PR0154119    2
PR0156928    2
PR0168063    2
Name: facility_address, dtype: int64

In [41]:
df_test[df_test['record_id']=='PR0030008']

,serial_number,activity_date,facility_name,score,grade,service_code,service_description,employee_id,facility_address,facility_city,facility_id,facility_state,facility_zip,owner_id,owner_name,pe_description,program_element_pe,program_name,program_status,record_id
10955,DAQI22YCI,2017-03-14T00:00:00.000,BOSCO CAFE BAKERY,96,A,1,ROUTINE INSPECTION,EE0000854,928 S WESTERN AVE # #103,LOS ANGELES,FA0031088,CA,90006,OW0009719,DELIZIA INC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,BOSCO CAFE BAKERY,ACTIVE,PR0030008
18243,DA0KDUY31,2016-09-06T00:00:00.000,BOSCO CAFE BAKERY,91,A,1,ROUTINE INSPECTION,EE0000854,928 S WESTERN AVE # #103,LOS ANGELES,FA0031088,CA,90006,OW0009719,DELIZIA INC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,BOSCO CAFE BAKERY,ACTIVE,PR0030008
35778,DAMHJJTSK,2015-09-02T00:00:00.000,BOSCO CAFE BAKERY,96,A,401,OWNER INITIATED ROUTINE INSPECT.,EE0000854,0928 S WESTERN AVE # #103,LOS ANGELES,FA0031088,CA,90006,OW0009719,DELIZIA INC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,BOSCO CAFE BAKERY,ACTIVE,PR0030008
36659,DAW0CPZOT,2015-08-17T00:00:00.000,BOSCO CAFE BAKERY,81,B,1,ROUTINE INSPECTION,EE0000854,0928 S WESTERN AVE # #103,LOS ANGELES,FA0031088,CA,90006,OW0009719,DELIZIA INC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,BOSCO CAFE BAKERY,ACTIVE,PR0030008
51789,DAMYC7OGM,2017-12-28T00:00:00.000,BOSCO CAFE BAKERY,96,A,1,ROUTINE INSPECTION,EE0000854,928 S WESTERN AVE # #103,LOS ANGELES,FA0031088,CA,90006,OW0009719,DELIZIA INC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,BOSCO CAFE BAKERY,ACTIVE,PR0030008
53324,DAPXM8EWR,2016-06-13T00:00:00.000,BOSCO CAFE BAKERY,90,A,1,ROUTINE INSPECTION,EE0000854,928 S WESTERN AVE # #103,LOS ANGELES,FA0031088,CA,90006,OW0009719,DELIZIA INC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,BOSCO CAFE BAKERY,ACTIVE,PR0030008
57281,DAX8QYZZW,2016-02-10T00:00:00.000,BOSCO CAFE BAKERY,91,A,1,ROUTINE INSPECTION,EE0000854,928 S WESTERN AVE # #103,LOS ANGELES,FA0031088,CA,90006,OW0009719,DELIZIA INC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,BOSCO CAFE BAKERY,ACTIVE,PR0030008
57985,DA0CQFV32,2018-05-16T00:00:00.000,BOSCO CAFE BAKERY,92,A,1,ROUTINE INSPECTION,EE0000854,928 S WESTERN AVE # #103,LOS ANGELES,FA0031088,CA,90006,OW0009719,DELIZIA INC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,BOSCO CAFE BAKERY,ACTIVE,PR0030008


In [63]:
a = df_test['facility_name'].unique()
print(len(a))
num = 0
for a_name in a:
    print(num)
    a_name = a_name.lower()
    for name in df_test['facility_name'][1:]:
        name = name.lower()
        ratio = fuzz.ratio(a_name, name.lower())
        if ratio > 85:
            print(a_name, name, ratio)
    num += 1

12071
0
('habitat coffee shop', 'habitat coffee shop', 100)
('habitat coffee shop', 'habitat coffee shop', 100)
('habitat coffee shop', 'habitat coffee shop', 100)
('habitat coffee shop', 'habitat coffee shop', 100)
('habitat coffee shop', 'habitat coffee shop', 100)
1
("reilly's", "reilly's", 100)
("reilly's", "reilly's", 100)
("reilly's", "reilly's", 100)
("reilly's", "reilly's", 100)
2
('street churros', 'street churros', 100)
('street churros', 'street churros', 100)
('street churros', 'street churros', 100)
3
('triniti echo park', 'triniti echo park', 100)
('triniti echo park', 'triniti echo park', 100)
4
('pollen', 'pollen', 100)
('pollen', 'pollen', 100)
5
('the spot grill', 'the spot grill', 100)
('the spot grill', 'the spot grill', 100)
('the spot grill', 'the spot grill', 100)
6
('7 eleven #37215a', '7 eleven #37215a', 100)
('7 eleven #37215a', '7-eleven #37621a', 88)
('7 eleven #37215a', '7-eleven #37175a', 88)
('7 eleven #37215a', '7-eleven #37175a', 88)
('7 eleven #37215a'

KeyboardInterrupt: 

In [55]:
a

'HABITAT COFFEE SHOP'

In [49]:
Ratio = fuzz.ratio(a.lower(), b.lower())

In [50]:
print(Ratio)

95


In [22]:
df_test.head(3)

,serial_number,activity_date,facility_name,score,grade,service_code,service_description,employee_id,facility_address,facility_city,facility_id,facility_state,facility_zip,owner_id,owner_name,pe_description,program_element_pe,program_name,program_status,record_id
0,DAJ00E07B,2017-12-29T00:00:00.000,HABITAT COFFEE SHOP,95,A,1,ROUTINE INSPECTION,EE0000923,3708 N EAGLE ROCK BLVD,LOS ANGELES,FA0170465,CA,90065,OW0178123,GLASSELL COFFEE SHOP LLC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,HABITAT COFFEE SHOP,ACTIVE,PR0160774
1,DAQOKRFZB,2017-12-29T00:00:00.000,REILLY'S,92,A,1,ROUTINE INSPECTION,EE0000633,100 WORLD WAY # 120,LOS ANGELES,FA0244690,CA,90045,OW0208441,"AREAS SKYVIEW LAX JV, LLC",RESTAURANT (0-30) SEATS MODERATE RISK,1631,REILLY'S,ACTIVE,PR0193026
2,DASJI4LUR,2017-12-29T00:00:00.000,STREET CHURROS,93,A,1,ROUTINE INSPECTION,EE0000835,6801 HOLLYWOOD BLVD # 253,LOS ANGELES,FA0224109,CA,90028,OW0228670,"STREETCHURROS, INC",RESTAURANT (0-30) SEATS LOW RISK,1630,STREET CHURROS,ACTIVE,PR0179282
